## Instruction how to deploy juvelery management and production system
* https://www.digitalocean.com/community/tutorials/how-to-set-up-django-with-postgres-nginx-and-gunicorn-on-ubuntu#step-2-creating-the-postgresql-database-and-user - main source
* OS - debian bookworm server
* database - postgresql
* Web framework - django
* Web server - nginx
* WSGI HTTP Server - gunicorn
* other tools - pandas, sqlalchemy

## Install postgresql
* https://wiki.debian.org/PostgreSql - source
* `sudo apt update` - update rep
* `sudo apt upgrade` - update system
* `sudo apt install postgresql postgresql-client postgresql-contrib libpq-dev` - install postgresql
* `sudo systemctl status postgresql.service` - check if service is running

## Configure database
* https://hostman.com/tutorials/installing-postgresql-on-debian/ - source
* `pip install pandas SQLAlchemy` - install needed python packages
* `sudo -u postgres psql` - run psql as an user postgres
* `CREATE DATABASE db_name;` - create database
* `CREATE USER admin WITH PASSWORD '12345';` - create admin role
* `ALTER ROLE user WITH LOGIN` - give user login permission if you use ROLE instead of USER during creation
* `ALTER DATABASE db_name OWNER TO admin;`
* add port 5432 to firewall
* `sudo nano /etc/postgresql/15/main/postgresql.conf` - edit config file to connect to postgresql from network
* `listen_addresses = '*'` - add/change this line (you don't need this if you will not conect to db from remote pc)
* `sudo nano /etc/postgresql/15/main/pg_hba.conf` - edit conncetion config file and add following line, put your IP (you don't need this if you will not conect to db from remote pc)
* `host   db_name   all   IP  scram-sha-256`
* `sudo systemctl status postgresql` - restart database
* `nmap 192.168.1.1 -p5432` - to check that port is open (change ip and port)

## Create tables and other database objects
Refer to  database.ipynb notebook

## Create Python Virtual Environment
* `mkdir ~/projects` - project direcotry
* `cd ~/projects`
* `sudo apt install python3-venv python3-dev`
* `python3 -m venv projectenv` - create environment
* `source projectenv/bin/activate` - activate environment

## Creating and Configuring a New Django Project
* `pip install django gunicorn psycopg2-binary cryptography python-dotenv SQLalchemy Pillow pandas` - install needed python packages
* `django-admin startproject project ~/projects` - create new project
* A Django project management script will be in this dir `~/projects/manage.py`
* project dir will be `~/projects/project/`
* generate `SECRET_KEY` using `from django.core.management.utils import get_random_secret_key` and `print(get_random_secret_key())`
* `nano ~/projects/.env` - create .env file and enter following info
    * ```
      SECRET_KEY=sssseeeeccccrrrreeeettttkkkkkeeeeyyyyyyy
      DEBUG=False
      DATABASE_NAME=db_name
      DATABASE_USER=admin
      DATABASE_PASSWORD=123456
      DATABASE_HOST=127.0.0.1
      DATABASE_PORT=5432
      ENCRYPTION_KEY=sssseeeeccccrrrreeeettttkkkkkeeeeyyyyyyy
      ```
* `chmod go-rwx .env` - leave only user access
* `nano ~/projects/project/settings.py` - change settings
    * ```
      # add/change settings
      import os
      from dotenv import load_dotenv
      # load .env
      load_dotenv()

      SECRET_KEY = os.getenv('SECRET_KEY')

      DEBUG = os.environ.get('DEBUG')

      # servers IP instead of `IP`
      ALLOWED_HOSTS = ['127.0.0.1', 'IP', 'localhost']

      DATABASES = {
            'default': {
                'ENGINE': 'django.db.backends.postgresql_psycopg2',
                'NAME': os.environ.get('DATABASE_NAME'),
                'USER': os.environ.get('DATABASE_USER'),
                'PASSWORD': os.environ.get('DATABASE_PASSWORD'),
                'HOST': os.environ.get('DATABASE_HOST'),
                'PORT': os.environ.get('DATABASE_PORT'),
            }
          }

      # according to your location
      TIME_ZONE = 'Asia/Tbilisi'

      MEDIA_ROOT = os.path.join(BASE_DIR, 'media/')
      MEDIA_URL = 'media/'
      STATIC_ROOT = os.path.join(BASE_DIR, 'static/')
      STATIC_URL = 'static/'
      ```
    * `mkdir static media`
    * `./manage.py makemigrations`
    * `./manage.py migrate` - migrate the initial database schema to our PostgreSQL database 
    * `./manage.py createsuperuser` - Create an administrative user
    * `./manage.py collectstatic` - collect all of the static content into the directory static
    * allow 8000 port in firewall
    * `./manage.py runserver 0.0.0.0:8000` - test server

## Testing Gunicorn
* `cd ~/projects` - go to projects dir
* `gunicorn --bind 0.0.0.0:8000 project.wsgi` - starts app, you should be able to open web page

## Configure systemd for Gunicorn
* `sudo nano /etc/systemd/system/gunicorn.socket` - create a systemd socket with following content
* ```
    [Unit]
    Description=gunicorn socket
    
    [Socket]
    ListenStream=/run/gunicorn.sock
    
    [Install]
    WantedBy=sockets.target
  ```
* `sudo nano /etc/systemd/system/gunicorn.service` - create systemd service file with following content
* ```
    [Unit]
    Description=gunicorn daemon
    Requires=gunicorn.socket
    After=network.target

    [Service]
    User=username
    Group=www-data
    WorkingDirectory=/home/username/projects
    ExecStart=/home/username/projects/projectenv/bin/gunicorn \
              --access-logfile - \
              --workers 3 \
              --bind unix:/run/gunicorn.sock \
              project.wsgi:application

    [Install]
    WantedBy=multi-user.target  
  ```
* `sudo systemctl start gunicorn.socket`
* `sudo systemctl enable gunicorn.socket`
* `sudo systemctl status gunicorn.socket` - check if socket is running
* `file /run/gunicorn.sock` - check socket is there
* `sudo journalctl -u gunicorn.socket` - check any logs, errors
* `curl --unix-socket /run/gunicorn.sock localhost` - send a connection to the socket through curl will activate gunicorn.serivece
* `sudo systemctl status gunicorn.service` - will show that service is active
* `sudo journalctl -u gunicorn` - check logs for details and errors
* `sudo systemctl daemon-reload` - reload daemon if you will change `/etc/systemd/system/gunicorn.service` file
* `sudo systemctl restart gunicorn`

## Configure Nginx https
* `sudo apt install nginx curl`
* `sudo nano /etc/nginx/sites-available/cauli` - create new server block with following contents
* ```
    server {
        listen 8000 default_server http2 ssl deferred;
        server_name 192.168.192.10;
        ssl_certificate /path/to/ssl/certs/file.crt;
        ssl_certificate_key /path/to/ssl/private/file.key;

        server_name 192.168.1.1;
    
        location = /favicon.ico { access_log off; log_not_found off; }
        location /static/ {
            root /home/username/projects;
        }
    
        location / {
            include proxy_params;
            proxy_pass http://unix:/run/gunicorn.sock;
        }
    }  
  ```
* `sudo ln -s /etc/nginx/sites-available/cauli /etc/nginx/sites-enabled/` - enable site
* `sudo nginx -t` - test for syntax errors
* `sudo systemctl restart nginx`
* static fails may not work if you have static folder under user dir and www-data can not access it, change project location or only static folder location to /var/www/
* `namei -l /home/username/projectlocation` - check permissions of dir tree

## Create ERP App
* `./manage.py startapp erp` - create app in project folder
* update settings.py
    * add `'erp',` to `INSTALLED_APPS`
    * add `'erp.full_auth_middleware.FullAuthMiddleware',` to `MIDDLEWARE` for automatic redirection to ligin page if not authorized. later add full_auth_middleware.py to app folder
* update urls.py
    * `from django.urls import include, path`
    * `from django.contrib import admin`
    * `path('', include('erp.urls')),` - add urls.py in erp folder
* `sudo systemctl restart gunicorn.service` - restart gunicorn each time you will update app

## Create db Tables
Refer to database.ipynb notebook

## Importing models from database
* `./manage.py inspectdb > erp/models.py`
* imported models need modification (see models.py for details)
* models should be registered in admin.py to be able to manage from admin/ built in interface